<a href="https://colab.research.google.com/github/nayankote/Neural-Style-Transfer/blob/master/Neural_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import imageio
import pprint
from PIL import Image
from tensorflow.python.framework import ops
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
class config:
    
    image_width = 800
    image_height = 600
    color_channels = 3
    noise_ratio = 0.6
    means = np.array([123.68, 116.779, 103.999]).reshape((1,1,1,3)) 
    #this is the mean RGB value of the pictures in the imagenet data
    #surprisingly(or not) a mud greyish brown is the most common color
 
    model = "/content/gdrive/My Drive/Colab Notebooks/Neural Style Transfer/imagenet-vgg-verydeep-19.mat"
    style_image = '/content/gdrive/My Drive/Colab Notebooks/Neural Style Transfer/monet.jpg'
    content_image = '/content/gdrive/My Drive/Colab Notebooks/Neural Style Transfer/louvre.jpg'

    #These are the weights given to each layer for the style error calculation, try changing them. 
    style_layers = [    
    ('conv1_1', 0.2),
    ('conv2_1', 0.2),
    ('conv3_1', 0.2),
    ('conv4_1', 0.2),
    ('conv5_1', 0.2)] 

In [0]:
def load_vgg_model():
    
    model_load = scipy.io.loadmat("/content/gdrive/My Drive/Colab Notebooks/Neural Style Transfer/imagenet-vgg-verydeep-19.mat")
    model_layers = model_load['layers']
    
    def _weights(layer, expected_layer_name):
        wb = model_layers[0][layer][0][0][2] 
        W = wb[0][0]
        b = wb[0][1]
        layer_name = model_layers[0][layer][0][0][0]
        assert layer_name == expected_layer_name
        return W,b
        
    def _conv2d(prev_layer, layer, layer_name):
        W,b = _weights(layer,layer_name)
        W = tf.constant(W)
        b = tf.constant(np.reshape(b, (b.size)))
        return tf.nn.conv2d(prev_layer, filters = W, strides = [1,1,1,1], padding = 'SAME') + b
    
    def _conv2d_relu(prev_layer, layer, layer_name):
        return tf.nn.relu(_conv2d(prev_layer, layer, layer_name))
    
    def _pool(prev_layer, pooltype = 'avg'):
        if pooltype == 'avg':
            return tf.nn.avg_pool(prev_layer, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
        else : 
            return tf.nn.max_pool(prev_layer, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
    
    graph = {}
    graph['input']   = tf.Variable(np.zeros((1, config.image_height, config.image_width, config.color_channels)), dtype = 'float32')
    graph['conv1_1']  = _conv2d_relu(graph['input'], 0, 'conv1_1')
    graph['conv1_2']  = _conv2d_relu(graph['conv1_1'], 2, 'conv1_2')
    graph['avgpool1'] = _pool(graph['conv1_2'])
    graph['conv2_1']  = _conv2d_relu(graph['avgpool1'], 5, 'conv2_1')
    graph['conv2_2']  = _conv2d_relu(graph['conv2_1'], 7, 'conv2_2')
    graph['avgpool2'] = _pool(graph['conv2_2'])
    graph['conv3_1']  = _conv2d_relu(graph['avgpool2'], 10, 'conv3_1')
    graph['conv3_2']  = _conv2d_relu(graph['conv3_1'], 12, 'conv3_2')
    graph['conv3_3']  = _conv2d_relu(graph['conv3_2'], 14, 'conv3_3')
    graph['conv3_4']  = _conv2d_relu(graph['conv3_3'], 16, 'conv3_4')
    graph['avgpool3'] = _pool(graph['conv3_4'])
    graph['conv4_1']  = _conv2d_relu(graph['avgpool3'], 19, 'conv4_1')
    graph['conv4_2']  = _conv2d_relu(graph['conv4_1'], 21, 'conv4_2')
    graph['conv4_3']  = _conv2d_relu(graph['conv4_2'], 23, 'conv4_3')
    graph['conv4_4']  = _conv2d_relu(graph['conv4_3'], 25, 'conv4_4')
    graph['avgpool4'] = _pool(graph['conv4_4'])
    graph['conv5_1']  = _conv2d_relu(graph['avgpool4'], 28, 'conv5_1')
    graph['conv5_2']  = _conv2d_relu(graph['conv5_1'], 30, 'conv5_2')
    graph['conv5_3']  = _conv2d_relu(graph['conv5_2'], 32, 'conv5_3')
    graph['conv5_4']  = _conv2d_relu(graph['conv5_3'], 34, 'conv5_4')
    graph['avgpool5'] = _pool(graph['conv5_4'])
    
    return graph

In [0]:
def resize_image(img):
    
    return img.resize((config.image_width,config.image_height), Image.ANTIALIAS)

In [0]:
def generate_noise_image(content_image, noise_ratio = config.noise_ratio):
    
    noise_image = np.random.uniform(-100, 100, (1, config.image_height, config.image_width, config.color_channels)).astype('float32') 
    #check what happens with np.random.randn and changing +- 20
    
    input_image = noise_ratio*noise_image + (1-noise_ratio)*content_image 
    #check what happens when noise ratio changes
    
    return input_image 

In [0]:
def reshape_and_normalize_image(image):
    
    image = np.reshape(image, ((1,) + image.shape)) #required input shape for vgg19

    image = image - config.means
    
    return image

In [0]:
def save_image(path, image):
    
    image = image + config.means
    
    image = np.clip(image[0],0,255).astype('uint8') 
    #changes values >255 (or <0) to 255 (or 0)
    
    imageio.imwrite(path, image)

In [0]:
#cost computation
def content_cost(a_C,a_G):
    (m, n_H, n_W, n_C) = a_G.get_shape().as_list()
    
    #a_C and a_G are unrolled from (m x n_H x n_W x n_C) to (m x (n_h*n_W) x n_C) 
    a_C_unrolled = tf.reshape(tf.transpose(a_C, perm = [0,3,1,2]), shape = [m,-1,n_C])
    a_G_unrolled = tf.reshape(tf.transpose(a_G, perm = [0,3,1,2]), shape = [m,-1,n_C])
    
    J_content = tf.reduce_sum(tf.square(tf.subtract(a_C,a_G)))/(4*n_H*n_W*n_C)
    
    return J_content

def gram_matrix(img):
    
    return tf.matmul(img,tf.transpose(img))

def style_cost_layer(a_S,a_G):
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    #a_S and a_G are unrolled from (m x n_H x n_W x n_C) to (n_C x (n_H*n_W)) [m = 1]  
    a_S_unrolled = tf.reshape(tf.transpose(a_S, perm = [0,3,1,2]), shape = [n_C,-1])
    a_G_unrolled = tf.reshape(tf.transpose(a_G, perm = [0,3,1,2]), shape = [n_C,-1])
    
    gram_a_S = gram_matrix(a_S_unrolled)
    gram_a_G = gram_matrix(a_G_unrolled)
    
    J_style_layer = tf.reduce_sum(tf.reduce_sum(tf.square(tf.subtract(gram_a_S, gram_a_G))))/((2*n_H*n_W*n_C)**2)
    
    return J_style_layer

def total_style_cost(model, layers = config.style_layers):
    
    J_style = 0
    
    for layer_name, coeff in layers:
        out = model[layer_name]
        
        a_S = sess.run(out)
        
        a_G = out
        
        J_style += coeff*style_cost_layer(a_S,a_G)
    
    return J_style

def total_cost(J_content, J_style, alpha = 10, beta = 40): #mess around with the weights
    
        return alpha*J_content + beta*J_style

In [0]:
# Reset the graph
ops.reset_default_graph()

In [0]:
#loading images 
content_image = Image.open(config.content_image)
content_image = np.array(resize_image(content_image))
content_image = reshape_and_normalize_image(content_image)
style_image = Image.open(config.style_image)
style_image = np.array(resize_image(style_image))
style_image = reshape_and_normalize_image(style_image)
generated_image = generate_noise_image(content_image)
imshow(generated_image[0])

In [0]:
# loading model 
model = load_vgg_model()

In [0]:
with tf.compat.v1.Session() as sess : 
    sess.run(model['input'].assign(content_image))
    out = model['conv4_2']
    a_C = sess.run(out)
    a_G = out
    J_content = content_cost(a_C,a_G)
    
    sess.run(model['input'].assign(style_image))
    J_style = total_style_cost(model,config.style_layers)
    
    #mess around with these weights
    J = total_cost(J_content,J_style,10,40)
    
    optimizer = tf.compat.v1.train.AdamOptimizer(2)
    train_step = optimizer.minimize(J)
    
    def model_nn(sess, input_image, num_iterations = 1000):
        sess.run(tf.compat.v1.global_variables_initializer())
        
        sess.run(model['input'].assign(input_image))
        
        for i in range(num_iterations):
            sess.run(train_step)
            
            generated_image = sess.run(model['input'])
            
            if i%200 == 0 :
                Jt,Jc,Js = sess.run([J, J_content, J_style])
                print("Iteration "+str(i) +":")
                print("total cost = "+str(Jt))
                print("content cost = "+str(Jc))
                print("style cost = "+str(Js))
                save_image("/content/gdrive/My Drive/Colab Notebooks/Neural Style Transfer/"+str(i) +".png",generated_image)
            
        save_image('/content/gdrive/My Drive/Colab Notebooks/Neural Style Transfer/generated_image_final.jpg',generated_image)
        
        return generated_image
    
    model_nn(sess, generated_image)